# Импрот библиотек

In [ ]:
import pandas as pd
import tqdm
import folium
import geopandas as gpd
from shapely.geometry import Point

# Загружаем данные с координатами всех билбордов из обучающего датасета

In [ ]:
bills = pd.read_excel('bill_coordinate.xlsx')

In [ ]:
bills

,lat,lon,azimuth
0,55.573691,37.631423,273
1,55.584765,37.712454,232
2,55.808425,37.388808,188
3,55.674378,37.422364,333
4,55.608396,37.766383,54
...,...,...,...
11802,55.694607,37.676856,93
11803,55.776881,37.524088,40
11804,55.623283,37.421858,324
11805,55.826256,37.629043,114


Для дальнейшей работы необходимо перейти к числам с плавающей точкой

In [ ]:
bills['lat'] = bills['lat'].astype(float)
bills['lon'] = bills['lon'].astype(float)

Для более качественной разметки нашего датасета было решено разделить билборды по районам. Для этого мы нашли границы для каждого района Москвы и визулизировали все районы на карте

In [ ]:
gdf = gpd.read_file('ao.geojson')

In [ ]:
map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
map = folium.Map(location=map_center, zoom_start=10)
for _, row in gdf.iterrows():
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_json = folium.GeoJson(data=geo_j,
                              style_function=lambda x: {
                                  'fillColor': 'blue',
                                  'color': 'blue',
                                  'fillOpacity': 0.2
                              })
    geo_json.add_to(map)
map

/var/folders/zl/wtf6lxgn593_7tswz2pzs80m0000gn/T/ipykernel_4905/3251401293.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]


Теперь каждой паре координат, которая у нас есть, присвоимм округ

In [ ]:
# Пример загрузки данных
gdf_polygons = gpd.read_file('ao.geojson')  # Загрузка полигонов административных округов
df_billboards = pd.read_excel('bill_coordinate.xlsx')  # Загрузка данных билбордов

geometry = [Point(xy) for xy in zip(df_billboards['lon'], df_billboards['lat'])]
gdf_billboards = gpd.GeoDataFrame(df_billboards, geometry=geometry)

# Присвоение административного округа каждому билборду
def get_administrative_area(point, gdf_polygons):
    for idx, row in gdf_polygons.iterrows():
        if row['geometry'].contains(point):
            return row['NAME']
    return None

gdf_billboards['Административный округ'] = gdf_billboards['geometry'].apply(lambda x: get_administrative_area(x, gdf_polygons))

# Сохранение результата
gdf_billboards.to_csv('billboards_with_areas.csv', index=False)

# Отображение результата
print(gdf_billboards.head())

         lat        lon  azimuth                   geometry  \
0  55.573691  37.631423      273  POINT (37.63142 55.57369)   
1  55.584765  37.712454      232  POINT (37.71245 55.58476)   
2  55.808425  37.388808      188  POINT (37.38881 55.80843)   
3  55.674378  37.422364      333  POINT (37.42236 55.67438)   
4  55.608396  37.766383       54   POINT (37.76638 55.6084)   

  Административный округ  
0                  Южный  
1                  Южный  
2        Северо-Западный  
3               Западный  
4                   None  


In [ ]:
gdf_billboards

,lat,lon,azimuth,geometry,Административный округ
0,55.573691,37.631423,273,POINT (37.63142 55.57369),Южный
1,55.584765,37.712454,232,POINT (37.71245 55.58476),Южный
2,55.808425,37.388808,188,POINT (37.38881 55.80843),Северо-Западный
3,55.674378,37.422364,333,POINT (37.42236 55.67438),Западный
4,55.608396,37.766383,54,POINT (37.76638 55.6084),None
...,...,...,...,...,...
11802,55.694607,37.676856,93,POINT (37.67686 55.69461),Южный
11803,55.776881,37.524088,40,POINT (37.52409 55.77688),Северный
11804,55.623283,37.421858,324,POINT (37.42186 55.62328),Новомосковский
11805,55.826256,37.629043,114,POINT (37.62904 55.82626),Северо-Восточный


In [ ]:
gdf_billboards

,lat,lon,azimuth,geometry,Административный округ
0,55.573691,37.631423,273,POINT (37.63142 55.57369),Южный
1,55.584765,37.712454,232,POINT (37.71245 55.58476),Южный
2,55.808425,37.388808,188,POINT (37.38881 55.80843),Северо-Западный
3,55.674378,37.422364,333,POINT (37.42236 55.67438),Западный
4,55.608396,37.766383,54,POINT (37.76638 55.6084),None
...,...,...,...,...,...
11802,55.694607,37.676856,93,POINT (37.67686 55.69461),Южный
11803,55.776881,37.524088,40,POINT (37.52409 55.77688),Северный
11804,55.623283,37.421858,324,POINT (37.42186 55.62328),Новомосковский
11805,55.826256,37.629043,114,POINT (37.62904 55.82626),Северо-Восточный


Проверка на уникальные административные районы Москвы (чтобы не попало ничего лишнего и ничего не потерялось)

In [ ]:
gdf_billboards['Административный округ'].unique()

array(['Южный', 'Северо-Западный', 'Западный', None, 'Северо-Восточный',
       'Юго-Западный', 'Восточный', 'Северный', 'Юго-Восточный',
       'Центральный', 'Новомосковский', 'Троицкий', 'Зеленоградский'],
      dtype=object)

В ходе работы с данными мы обратили внимание на то, что у части точек остался непрсовенный округ. После анализа местоположения билбордов, мы заметили, что все неклассифицированные билборды находятся в Подмосковье. Поэтому мы решили заполнить все пропуски, поставив Подмосковье  

In [ ]:
gdf_billboards['Административный округ'].fillna("Подмосковье", inplace=True)

/var/folders/zl/wtf6lxgn593_7tswz2pzs80m0000gn/T/ipykernel_4905/3648792507.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf_billboards['Административный округ'].fillna("Подмосковье", inplace=True)


In [ ]:
df = pd.DataFrame(gdf_billboards)
df

,lat,lon,azimuth,geometry,Административный округ
0,55.573691,37.631423,273,POINT (37.63142 55.57369),Южный
1,55.584765,37.712454,232,POINT (37.71245 55.58476),Южный
2,55.808425,37.388808,188,POINT (37.38881 55.80843),Северо-Западный
3,55.674378,37.422364,333,POINT (37.42236 55.67438),Западный
4,55.608396,37.766383,54,POINT (37.76638 55.6084),Подмосковье
...,...,...,...,...,...
11802,55.694607,37.676856,93,POINT (37.67686 55.69461),Южный
11803,55.776881,37.524088,40,POINT (37.52409 55.77688),Северный
11804,55.623283,37.421858,324,POINT (37.42186 55.62328),Новомосковский
11805,55.826256,37.629043,114,POINT (37.62904 55.82626),Северо-Восточный


In [ ]:
df.drop(columns = 'geometry' , inplace = True)

In [ ]:
df.dtypes

lat                       float64
lon                       float64
azimuth                     int64
Административный округ     object
dtype: object

Теперь, для более качественной разметки данных, было принято решение присвоить каждой паре координат не только округ, но и район  

Для этого мы нашли координаты границ каждого района Москвы

In [ ]:
data = pd.read_excel('moscow_admin_division_wth_ao.xlsx')
data

,Субъект,min_longitude,min_latitude,max_longitude,max_latitude
0,Арбат,37.572060,55.743712,37.612228,55.758061
1,Басманный,37.627940,55.750749,37.713525,55.781956
2,Замоскворечье,37.615133,55.720280,37.654211,55.749153
3,Красносельский,37.628386,55.759797,37.688804,55.792981
4,Мещанский,37.612223,55.759120,37.644896,55.797140
...,...,...,...,...,...
120,Крюково,37.144003,55.947901,37.220825,55.999244
121,Матушкино,37.177540,55.986998,37.231254,56.018630
122,Савёлки,37.209283,55.966915,37.268406,56.004668
123,Силино,37.131448,55.980439,37.205714,56.021208


In [ ]:
data['min_latitude'] = data['min_latitude'].astype(float)
data['max_latitude'] = data['max_latitude'].astype(float)
data['min_longitude'] = data['min_longitude'].astype(float)
data['max_longitude'] = data['max_longitude'].astype(float)

Теперь для каждой пары наших точек присваиваем район

In [ ]:
def get_district(lat, lon, data):
    for index, row in data.iterrows():
        if row['min_latitude'] <= lat <= row['max_latitude'] and row['min_longitude'] <= lon <= row['max_longitude']:
            return row['Субъект']
    return None

# Apply the function to each row in df with tqdm progress bar
#tqdm.pandas()
df['district'] = df.apply(lambda row: get_district(row['lat'], row['lon'], data), axis=1)

In [ ]:
df

,lat,lon,azimuth,Административный округ,district
0,55.573691,37.631423,273,Южный,Бирюлёво Западное
1,55.584765,37.712454,232,Южный,Бирюлёво Восточное
2,55.808425,37.388808,188,Северо-Западный,Кунцево
3,55.674378,37.422364,333,Западный,Очаково-Матвеевское
4,55.608396,37.766383,54,Подмосковье,Орехово-Борисово Южное
...,...,...,...,...,...
11802,55.694607,37.676856,93,Южный,Печатники
11803,55.776881,37.524088,40,Северный,Хорошёвский
11804,55.623283,37.421858,324,Новомосковский,None
11805,55.826256,37.629043,114,Северо-Восточный,Останкинский


И так же, как в ситуации с округом, заменяем пропуски на Подмосковье (данный вывод мы полностью подтвердили, визуализировав все неклассифицированные точки)

In [ ]:
df['district'].fillna("Подмосковье", inplace=True)

/var/folders/zl/wtf6lxgn593_7tswz2pzs80m0000gn/T/ipykernel_4905/2511637939.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['district'].fillna("Подмосковье", inplace=True)


# Влияние цены недвижимости

Нами была выдвинута гипотеза о том, что средняя стоимость жилья в том или ином районе оказывает непосредственное влияние на стоимость и количество просмотров. Поэтому мы взяли дынные по средней стоимости жилья для каждого района и применили к нашим данным для улучшения качества модели

In [ ]:
nedv = pd.read_excel('недвижимость.xlsx')

In [ ]:
nedv.rename(columns={'Субъект': 'district'}, inplace=True)

In [ ]:
nedv

,district,Стоимость недвижимости
0,Арбат,507328
1,Басманный,327820
2,Замоскворечье,396450
3,Красносельский,384762
4,Мещанский,396696
...,...,...
120,Крюково,204297
121,Матушкино,204297
122,Савёлки,204297
123,Силино,204297


In [ ]:
df = pd.merge(df, nedv, on='district', how='left')

In [ ]:
df

,lat,lon,azimuth,Административный округ,district,Стоимость недвижимости
0,55.573691,37.631423,273,Южный,Бирюлёво Западное,204613.0
1,55.584765,37.712454,232,Южный,Бирюлёво Восточное,204613.0
2,55.808425,37.388808,188,Северо-Западный,Кунцево,289554.0
3,55.674378,37.422364,333,Западный,Очаково-Матвеевское,288025.0
4,55.608396,37.766383,54,Подмосковье,Орехово-Борисово Южное,231958.0
...,...,...,...,...,...,...
11802,55.694607,37.676856,93,Южный,Печатники,256271.0
11803,55.776881,37.524088,40,Северный,Хорошёвский,313613.0
11804,55.623283,37.421858,324,Новомосковский,Подмосковье,NaN
11805,55.826256,37.629043,114,Северо-Восточный,Останкинский,279405.0


Так как в данных пристуствуют пропуски, нами была проделана аналитическая работа по нахождению оптимальной стоимости, которой мы заполним наши nan. Было выбрано число: 200000, поскольку большинство координат без присваивания цены опять были в Подмосковье

In [ ]:
df['Стоимость недвижимости'] = df['Стоимость недвижимости'].fillna(200000)

In [ ]:
df

,lat,lon,azimuth,Административный округ,district,Стоимость недвижимости
0,55.573691,37.631423,273,Южный,Бирюлёво Западное,204613.0
1,55.584765,37.712454,232,Южный,Бирюлёво Восточное,204613.0
2,55.808425,37.388808,188,Северо-Западный,Кунцево,289554.0
3,55.674378,37.422364,333,Западный,Очаково-Матвеевское,288025.0
4,55.608396,37.766383,54,Подмосковье,Орехово-Борисово Южное,231958.0
...,...,...,...,...,...,...
11802,55.694607,37.676856,93,Южный,Печатники,256271.0
11803,55.776881,37.524088,40,Северный,Хорошёвский,313613.0
11804,55.623283,37.421858,324,Новомосковский,Подмосковье,200000.0
11805,55.826256,37.629043,114,Северо-Восточный,Останкинский,279405.0


In [ ]:
quantiles = df['Стоимость недвижимости'].quantile([0.2, 0.4, 0.6 , 0.8])
quantiles

0.2    222055.0
0.4    248837.0
0.6    283692.0
0.8    357036.0
Name: Стоимость недвижимости, dtype: float64

In [ ]:
bins = [float('-inf'), quantiles[0.2], quantiles[0.4], quantiles[0.6], quantiles[0.8], float('inf')]
labels = ['Дешево', 'Ниже среднего', 'Средне', 'Дорого', 'Очень дорого']
df['Группа стоимости'] = pd.cut(df['Стоимость недвижимости'], bins=bins, labels=labels)

In [ ]:
df.rename(columns = {'district': "Район"} , inplace = True)

In [ ]:
df.drop(columns = 'Стоимость недвижимости', inplace = True)

In [ ]:
df.to_excel('for_back.xlsx' , index = False)

In [ ]:
df.isna().sum()

lat                       0
lon                       0
azimuth                   0
Административный округ    0
Район                     0
Группа стоимости          0
dtype: int64

# Финальный датасет

Таким образом, мы получили наш финальный расширенный датасет, на котором можно обучать нашу модель

In [ ]:
df

,lat,lon,azimuth,Административный округ,Район,Группа стоимости
0,55.573691,37.631423,273,Южный,Бирюлёво Западное,Дешево
1,55.584765,37.712454,232,Южный,Бирюлёво Восточное,Дешево
2,55.808425,37.388808,188,Северо-Западный,Кунцево,Дорого
3,55.674378,37.422364,333,Западный,Очаково-Матвеевское,Дорого
4,55.608396,37.766383,54,Подмосковье,Орехово-Борисово Южное,Ниже среднего
...,...,...,...,...,...,...
11802,55.694607,37.676856,93,Южный,Печатники,Средне
11803,55.776881,37.524088,40,Северный,Хорошёвский,Дорого
11804,55.623283,37.421858,324,Новомосковский,Подмосковье,Дешево
11805,55.826256,37.629043,114,Северо-Восточный,Останкинский,Средне
